# Load Libraries

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import os
import json
import csv

import pandas as pd
import numpy as np

from tqdm import tqdm
from collections import Counter
from pprint import pprint

from functions.functions_features import split_audio, mean_variance_normalize, process_row, pad_array, process_CNN_row

# Load data

In [ ]:
df_all = pd.read_csv('Results/Data/data_all.csv')
df_all

In [ ]:
# Get columns
columns = [
    'dataset', 'filepath', 'filename', 
    'age', 'gender', 'label', 'status', 'duration', 
    'duration_segment', 'sample_frequency', 'mean_amplitude',
    'segment_shape',
    ]
    
print(columns)

In [ ]:
list_dataset_name = [
    'coswara', 
    'coughvid', 
    'esc50', 
    'fsdkaggle', 
    'virufy',
    ]

overlap=0 # To avoid overfitting when doing kfold

In [ ]:
if not os.path.exists(f'Results/Features_CNN'):
    os.makedirs(f'Results/Features_CNN')

for segment_length in [1, 5, 10]:
    for dataset_name in list_dataset_name:
        print(f'\n{dataset_name} {segment_length}')
        
        df_all = pd.read_csv(f'Results/Data/data_summary_{dataset_name}.csv')
        df_all = df_all.sample(frac=1).groupby('label').head(1000).reset_index(drop=True)
        
        results_all = []
 
        path_save = f'Results/Features_CNN/data_{dataset_name}_features_{segment_length}s_{overlap}.csv'

        if os.path.exists(path_save) == False:
        # if True:
            for i in tqdm(range(len(df_all))):
                results, output_shape = process_CNN_row(i, df_all, segment_length, overlap)
                results_all.extend(results)
                
            print(output_shape)
            dimension_dictionary = {
                1: 22,
                5: 27,
                10: 27,
            }
            output_shape = dimension_dictionary[segment_length] * 128
            columns_additional = list(range(output_shape))
            results_all = pd.DataFrame(results_all, columns=columns + columns_additional)
            results_all.to_csv(path_save, index=False)
            print(results_all.shape)